In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

In [2]:
es = Elasticsearch("https://elasticsearch.ceda.ac.uk:443")

In [3]:
fixes = {}

for item in scan(es, index="c3s-roocs-fix-prop"):
    fixes[item["_source"]["dataset_id"]] = item["_source"]["fixes"]

In [4]:
len(fixes.keys())

13960

In [5]:
ds_id = list(fixes.keys())[0]
ds_id

'c3s-cmip6.DCPP.EC-Earth-Consortium.EC-Earth3.dcppA-hindcast.s2014-r7i2p1f1.day.pr.gr.v20200508'

In [6]:
fix = fixes[ds_id]
fix

[{'fix': {'fix_id': 'VarAttrFix',
   'title': 'Apply Fix to Attributes of any Variable',
   'description': '\nEdits or adds the attributes of a given variables e.g. fixing standard name or adding missing standard name\n\nTakes a dictionary of fixes with each fix as a key and value pair with the attribute\nto be changed as the key and what the value should be as the value.\n\nFor example:\n  - inputs:\n    {"var_id": "lev",\n    "attrs":\n        {"long_name": "Dissolved Oxygen Concentration",\n        "standard_name": "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water"}\n    }\n',
   'category': 'attr_fixes',
   'reference_implementation': 'daops.data_utils.attr_utils.edit_var_attrs',
   'process_type': 'post_processor',
   'operands': {'var_id': 'time', 'attrs': {'long_name': 'valid_time'}},
   'source': {'name': 'ceda',
    'version': '',
    'comments': '',
    'url': 'https://github.com/cp4cds/c3s34g_master/tree/master/Decadal'}},
  'status': 'published',
  'timestamp':

In [7]:
from daops.data_utils.attr_utils import edit_var_attrs, edit_global_attrs, remove_coord_attr
from daops.data_utils.coord_utils import add_scalar_coord, add_coord
from daops.data_utils.var_utils import add_data_var
import xarray as xr
from roocs_utils.xarray_utils.xarray_utils import open_xr_dataset

In [8]:
ds = open_xr_dataset(
    "/Users/pingu/tmp/c3s-cmip6.DCPP.EC-Earth-Consortium.EC-Earth3.dcppA-hindcast.s2014-r7i2p1f1.day.pr.gr.v20201216/*.nc")

ds

INFO:/usr/local/Caskroom/mambaforge/base/envs/rook/lib/python3.10/site-packages/roocs_utils/project_utils.py:The project could not be identified


<xarray.Dataset>
Dimensions:    (time: 4018, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) object 2014-11-01 12:00:00 ... 2025-10-31 12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(365, 256, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(365, 512, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(365, 256, 512), meta=np.ndarray>
Attributes: (12/48)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               DCPP
    branch_method:             no parent
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     0.0
    ...                        ...
    variable_id:               pr
    variant_label:             r7i1p1f1
    license:                   CMIP6 model data produced by EC-Earth-Consorti...
    cmor_version:              3.4.0
    variant_info:              Atmosphere initialization based on full-fields...
    history:                   Sat Sep 28 00:54:19 2019: ncatted -O -a varian...

In [9]:
# fix 1

operands = {'var_id': 'time', 'attrs': {'long_name': 'valid_time'}}
ds_mod = edit_var_attrs(ds_id, ds, **operands)

ds_mod

<xarray.Dataset>
Dimensions:    (time: 4018, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) object 2014-11-01 12:00:00 ... 2025-10-31 12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(365, 256, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(365, 512, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(365, 256, 512), meta=np.ndarray>
Attributes: (12/48)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               DCPP
    branch_method:             no parent
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     0.0
    ...                        ...
    variable_id:               pr
    variant_label:             r7i1p1f1
    license:                   CMIP6 model data produced by EC-Earth-Consorti...
    cmor_version:              3.4.0
    variant_info:              Atmosphere initialization based on full-fields...
    history:                   Sat Sep 28 00:54:19 2019: ncatted -O -a varian...

In [10]:
# fix 2
operands = {
    'attrs': {
        'forcing_description': 'derive: daops.fix_utils.decadal_utils.get_decadal_model_attr_from_dict: forcing_description',
        'physics_description': 'derive: daops.fix_utils.decadal_utils.get_decadal_model_attr_from_dict: physics_description',
        'initialization_description': 'derive: daops.fix_utils.decadal_utils.get_decadal_model_attr_from_dict: initialization_description',
        'startdate': 'derive: daops.fix_utils.decadal_utils.get_sub_experiment_id',
        'sub_experiment_id': 'derive: daops.fix_utils.decadal_utils.get_sub_experiment_id'}}

ds_mod = edit_global_attrs(ds_id, ds_mod, **operands)

ds_mod


<xarray.Dataset>
Dimensions:    (time: 4018, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) object 2014-11-01 12:00:00 ... 2025-10-31 12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(365, 256, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(365, 512, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(365, 256, 512), meta=np.ndarray>
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time:                 0.0
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    ...                          ...
    variant_info:                Atmosphere initialization based on full-fiel...
    history:                     Sat Sep 28 00:54:19 2019: ncatted -O -a vari...
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  Atmosphere initialization based on full-fiel...
    startdate:                   s201411

In [11]:
# fix 3

operands = {
    'var_id': 'reftime',
    'value': 'derive: daops.fix_utils.decadal_utils.get_reftime',
    'dtype': 'datetime64[ns]',
    'attrs': {
        'long_name': 'Start date of the forecast',
         'standard_name': 'forecast_reference_time'},
    'encoding': {'dtype': 'int32',
     'units': 'days since 1850-01-01',
     'calendar': 'derive: daops.fix_utils.decadal_utils.get_time_calendar'}}

ds_mod = add_scalar_coord(ds_id, ds_mod, **operands)

ds_mod

<xarray.Dataset>
Dimensions:    (time: 4018, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) object 2014-11-01 12:00:00 ... 2025-10-31 12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
    reftime    datetime64[ns] 2014-11-01
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(365, 256, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(365, 512, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(365, 256, 512), meta=np.ndarray>
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time:                 0.0
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    ...                          ...
    variant_info:                Atmosphere initialization based on full-fiel...
    history:                     Sat Sep 28 00:54:19 2019: ncatted -O -a vari...
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  Atmosphere initialization based on full-fiel...
    startdate:                   s201411

In [12]:
# fix 4

operands = {'var_id': 'leadtime',
    'value': 'derive: daops.fix_utils.decadal_utils.get_lead_times',
    'dim': ['time'],
    'dtype': 'float64',
    'attrs': {'long_name': 'Time elapsed since the start of the forecast',
     'standard_name': 'forecast_period',
     'units': 'days'},
    'encoding': {'dtype': 'double'}}

ds_mod = add_coord(ds_id, ds_mod, **operands)

ds_mod

<xarray.Dataset>
Dimensions:    (time: 4018, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) object 2014-11-01 12:00:00 ... 2025-10-31 12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
    reftime    datetime64[ns] 2014-11-01
    leadtime   (time) float64 0.0 1.0 2.0 3.0 ... 4.015e+03 4.016e+03 4.017e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(365, 256, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(365, 512, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(365, 256, 512), meta=np.ndarray>
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time:                 0.0
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    ...                          ...
    variant_info:                Atmosphere initialization based on full-fiel...
    history:                     Sat Sep 28 00:54:19 2019: ncatted -O -a vari...
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  Atmosphere initialization based on full-fiel...
    startdate:                   s201411

In [13]:
# fix 5
operands = {
    'var_id': 'realization',
    'value': '1',
    'dtype': 'int32',
    'attrs': {'long_name': 'realization',
    'comment': 'For more information on the ripf, refer to the variant_label, initialization_description, physics_description and forcing_description global attributes'}}

ds_mod = add_data_var(ds_id, ds_mod, **operands)

ds_mod


<xarray.Dataset>
Dimensions:      (time: 4018, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time         (time) object 2014-11-01 12:00:00 ... 2025-10-31 12:00:00
  * lat          (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon          (lon) float64 0.0 0.7031 1.406 2.109 ... 357.9 358.6 359.3
    reftime      datetime64[ns] 2014-11-01
    leadtime     (time) float64 0.0 1.0 2.0 ... 4.015e+03 4.016e+03 4.017e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (time, bnds) object dask.array<chunksize=(365, 2), meta=np.ndarray>
    lat_bnds     (time, lat, bnds) float64 dask.array<chunksize=(365, 256, 2), meta=np.ndarray>
    lon_bnds     (time, lon, bnds) float64 dask.array<chunksize=(365, 512, 2), meta=np.ndarray>
    pr           (time, lat, lon) float32 dask.array<chunksize=(365, 256, 512), meta=np.ndarray>
    realization  int32 1
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time:                 0.0
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    ...                          ...
    variant_info:                Atmosphere initialization based on full-fiel...
    history:                     Sat Sep 28 00:54:19 2019: ncatted -O -a vari...
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  Atmosphere initialization based on full-fiel...
    startdate:                   s201411